# Visualization Draft

**MOAZ**

---

## All Imports




In [14]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
from pathlib import Path

from data.loader import load_excel_sheets
from analysis.supply_demand import (
    compute_subcategory_supply, 
    compute_subcategory_demand, 
    merge_supply_demand
)
from analysis.gap_score import (
    compute_gap_score, 
    normalize_gap_score, 
    classify_gap_level,
    compute_demand_supply_ratio,
    rank_categories_by_gap
)
from visualization.plotly.interactive_charts import (
    create_supply_demand_chart,
    create_gap_score_severity_chart,
    create_gap_heatmap,
    create_category_ranking_chart,
    create_gap_distribution_pie
)
from visualization.dashboards.builder import calculate_kpis, create_kpi_cards
from insights.chart_insights import (
    generate_supply_demand_insight,
    generate_severity_insight,
    generate_heatmap_insight,
    generate_ranking_insight,
    generate_executive_summary
)
from data.exporter import export_dataframe_to_files, export_plotly_chart, export_insights_to_text

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

print("Environment configured successfully")

Environment configured successfully


---

##  Data Loading and Preparation



In [15]:
file_path = "../data/processed/Cleaned_data.xlsx"

product_lookup = pd.read_excel(file_path, sheet_name=' Product Lookup')
product_subcategory = pd.read_excel(file_path, sheet_name='Product Subcategory')
product_category = pd.read_excel(file_path, sheet_name='Product Category')
sales_data = pd.read_excel(file_path, sheet_name='Sales Data')

products_with_subcat = product_lookup.merge(product_subcategory, on='ProductSubcategoryKey', how='left')
products_full = products_with_subcat.merge(product_category, on='ProductCategoryKey', how='left')

sales_full = sales_data.merge(
    products_full[['ProductKey', 'CategoryName', 'SubcategoryName']],
    on='ProductKey',
    how='left'
)

print(f"Data loaded successfully:")
print(f"  Categories: {sales_full['CategoryName'].nunique()}")
print(f"  Subcategories: {sales_full['SubcategoryName'].nunique()}")
print(f"  Products: {len(product_lookup):,}")
print(f"  Sales transactions: {len(sales_data):,}")

Data loaded successfully:
  Categories: 3
  Subcategories: 17
  Products: 293
  Sales transactions: 56,046


---
## Gap Score Calculation 

Compute supply and demand metrics at **subcategory level**, then calculate gap scores with severity classification.

In [16]:
supply_df = compute_subcategory_supply(sales_data, products_full)

demand_df = compute_subcategory_demand(sales_full)


supply_demand_df = merge_supply_demand(supply_df, demand_df)

gap_df = compute_gap_score(supply_demand_df, demand_col='TotalQuantitySold', supply_col='UniqueProducts')
gap_df = normalize_gap_score(gap_df, gap_col='GapScore')
gap_df = classify_gap_level(gap_df, gap_col='GapScore')
gap_df = compute_demand_supply_ratio(gap_df)
gap_df = rank_categories_by_gap(gap_df, gap_col='GapScore', ascending=False)

gap_summary = gap_df[['CategoryName', 'SubcategoryName', 'UniqueProducts', 
                       'TotalQuantitySold', 'GapScore', 'GapLevel']].copy()
gap_summary.columns = ['parent_category', 'category', 'supply', 'demand', 'gap_score', 'gap_status']
gap_summary = gap_summary.sort_values('gap_score', ascending=False).reset_index(drop=True)

# BEST PRACTICE: Use ratio instead of percentage for extreme values
gap_summary['demand_supply_ratio'] = (gap_summary['demand'] / (gap_summary['supply'] + 1)).round(1)

# Only calculate percentage for moderate gaps (ratio < 10)
def calc_gap_metric(row):
    ratio = row['demand'] / (row['supply'] + 1)
    if ratio < 10:
        return f"{((row['demand'] - row['supply']) / (row['supply'] + 1) * 100):.1f}%"
    else:
        return f"{ratio:.1f}:1 ratio"

gap_summary['gap_metric'] = gap_summary.apply(calc_gap_metric, axis=1)

print("\nGap analysis completed:")
print(f"  Total subcategories: {len(gap_summary)}")
print(f"  Average gap score: {gap_summary['gap_score'].mean():.2f}")
print(f"  Highest gap: {gap_summary.iloc[0]['category']} ({gap_summary.iloc[0]['parent_category']})")
print(f"    Gap score: {gap_summary.iloc[0]['gap_score']:.2f}")
print(f"  Lowest gap: {gap_summary.iloc[-1]['category']} ({gap_summary.iloc[-1]['parent_category']})")
print(f"    Gap score: {gap_summary.iloc[-1]['gap_score']:.2f}")

display(gap_summary.head(15))


Gap analysis completed:
  Total subcategories: 37
  Average gap score: 402.16
  Highest gap: Bottles and Cages (Accessories)
    Gap score: 3776.75
  Lowest gap: Road Frames (Components)
    Gap score: 0.03


,parent_category,category,supply,demand,gap_score,gap_status,demand_supply_ratio,gap_metric
0,Accessories,Bottles and Cages,3,15106,"3,776.75",Critical Gap,"3,776.50",3776.5:1 ratio
1,Accessories,Tires and Tubes,11,29772,"2,481.08",Critical Gap,"2,481.00",2481.0:1 ratio
2,Clothing,Caps,1,4151,"2,076.00",Critical Gap,"2,075.50",2075.5:1 ratio
3,Accessories,Fenders,1,3960,"1,980.50",Critical Gap,"1,980.00",1980.0:1 ratio
4,Accessories,Helmets,3,6034,"1,508.75",Critical Gap,"1,508.50",1508.5:1 ratio
5,Accessories,Cleaners,1,1706,853.50,Critical Gap,853.00,853.0:1 ratio
6,Clothing,Gloves,6,2644,377.86,Critical Gap,377.70,377.7:1 ratio
7,Accessories,Hydration Packs,1,695,348.00,Critical Gap,347.50,347.5:1 ratio
8,Clothing,Jerseys,8,3113,346.00,Critical Gap,345.90,345.9:1 ratio
9,Clothing,Socks,4,1063,212.80,Critical Gap,212.60,212.6:1 ratio


---

##  Core Visualizations 

###  Supply vs Demand Comparison 


In [17]:
top_gaps = gap_summary.head(15).copy()

top_gaps['category_display'] = (
    top_gaps['category'] + '\n(' + top_gaps['parent_category'] + ')'
)
top_gaps['category'] = top_gaps['category_display']

chart_supply_demand = create_supply_demand_chart(top_gaps)
chart_supply_demand.update_layout(
    title=dict(text='<b>Supply vs Demand by Subcategory (Top 15 Gaps)</b><br><sub>Independent Y-axes for clear comparison</sub>')
)
chart_supply_demand.show()

insight_supply_demand = generate_supply_demand_insight(gap_summary)
print("\nKey Insight:")
print(insight_supply_demand)


Key Insight:
Bottles and Cages: 15106 demand vs 3 supply (ratio: 3776.5:1)


---

###  Gap Score Severity Analysis 

In [18]:
gap_chart_data = gap_summary.copy()
gap_chart_data['category_full'] = (
    gap_chart_data['category'] + ' (' + gap_chart_data['parent_category'] + ')'
)
gap_chart_data['category'] = gap_chart_data['category_full']

chart_severity = create_gap_score_severity_chart(gap_chart_data)
chart_severity.update_layout(
    title=dict(text='<b>Gap Score Severity Analysis (Subcategories)</b><br><sub>Color-coded by severity level</sub>'),
    height=max(600, len(gap_chart_data) * 20) 
)
chart_severity.show()

insight_severity = generate_severity_insight(gap_summary)
print("\nKey Insight:")
print(insight_severity)


Key Insight:
16/37 critical/high (43.2%). worst: Bottles and Cages (3776.75)


---

### Normalized Metrics Heatmap 


In [19]:
top_heatmap = gap_summary.head(20).copy()
top_heatmap['category_display'] = (
    top_heatmap['category'] + '\n(' + top_heatmap['parent_category'] + ')'
)
top_heatmap['category'] = top_heatmap['category_display']

chart_heatmap = create_gap_heatmap(top_heatmap)
chart_heatmap.update_layout(
    title=dict(text='<b>Gap Analysis Heatmap (Top 20 Subcategories)</b><br><sub>Normalized Metrics (0-1 Scale)</sub>'),
    height=500
)
chart_heatmap.show()

insight_heatmap = generate_heatmap_insight(gap_summary)
print("\nKey Insight:")
print(insight_heatmap)


Key Insight:
best balanced: Mountain Frames (gap: 0.03)


---

### Subcategory Rankings

In [20]:
ranking_data = gap_summary.copy()
ranking_data['category_full'] = (
    ranking_data['category'] + ' (' + ranking_data['parent_category'] + ')'
)
ranking_data['category'] = ranking_data['category_full']

chart_ranking = create_category_ranking_chart(ranking_data, top_n=10)
chart_ranking.update_layout(
    title=dict(text='<b>Subcategory Rankings: Highest & Lowest Gaps</b>'),
    height=600
)
chart_ranking.show()

insight_ranking = generate_ranking_insight(gap_summary, top_n=10)
print("\nKey Insight:")
print(insight_ranking)


Key Insight:
top 10 gaps: 212.80 to 3776.75


---

### Gap Severity Distribution 

In [21]:
chart_distribution = create_gap_distribution_pie(gap_summary)
chart_distribution.update_layout(
    title=dict(text='<b>Gap Severity Distribution (Subcategories)</b>')
)
chart_distribution.show()

gap_level_counts = gap_summary['gap_status'].value_counts()
print("\nGap Level Distribution:")
for level, count in gap_level_counts.items():
    print(f"  {level}: {count} subcategories ({count/len(gap_summary)*100:.1f}%)")

print(f"\nCritical breakdown by parent category:")
critical_by_category = gap_summary[
    gap_summary['gap_status'].isin(['Critical Gap', 'High Gap'])
].groupby('parent_category').size().sort_values(ascending=False)
display(critical_by_category)


Gap Level Distribution:
  Low Gap: 20 subcategories (54.1%)
  Critical Gap: 10 subcategories (27.0%)
  High Gap: 6 subcategories (16.2%)
  Moderate Gap: 1 subcategories (2.7%)

Critical breakdown by parent category:


parent_category
Accessories    8
Clothing       6
Bikes          2
dtype: int64

---

##  Key Performance Indicators 

In [22]:
kpis = calculate_kpis(gap_summary)
kpi_fig = create_kpi_cards(kpis)
kpi_fig.update_layout(
    title=dict(text='<b>Subcategory Gap Analysis KPI Overview</b>')
)
kpi_fig.show()

print("\nKey Performance Indicators (Subcategory Level):")
print(f"  Average Gap Score: {kpis['avg_gap_score']}")
print(f"  Highest Gap Subcategory: {kpis['highest_gap_category']} ({kpis['highest_gap_value']})")
print(f"  Total Subcategories Analyzed: {kpis['total_categories']}")
print(f"  Subcategories with High/Critical Gap: {kpis['critical_high_count']} ({kpis['high_gap_pct']}%)")
print(f"  Total Supply: {kpis['total_supply']:,} units")
print(f"  Total Demand: {kpis['total_demand']:,} units")

print("\n\nTop 5 Categories by Number of Critical/High Gap Subcategories:")
critical_subcats = gap_summary[
    gap_summary['gap_status'].isin(['Critical Gap', 'High Gap'])
].groupby('parent_category').agg({
    'category': 'count',
    'gap_score': 'mean'
}).rename(columns={'category': 'critical_subcat_count', 'gap_score': 'avg_gap_score'})
critical_subcats = critical_subcats.sort_values('critical_subcat_count', ascending=False)
display(critical_subcats.head())


Key Performance Indicators (Subcategory Level):
  Average Gap Score: 402.16
  Highest Gap Subcategory: Bottles and Cages (3776.75)
  Total Subcategories Analyzed: 37
  Subcategories with High/Critical Gap: 16 (43.2%)
  Total Supply: 293 units
  Total Demand: 84,174 units


Top 5 Categories by Number of Critical/High Gap Subcategories:


,critical_subcat_count,avg_gap_score
parent_category,,
Accessories,8,"1,402.20"
Clothing,6,543.55
Bikes,2,152.00


---

 ## Export Deliverables

In [23]:
output_base = Path("../results/visualizations_subcategory")
output_base.mkdir(parents=True, exist_ok=True)

print("Exporting subcategory-level deliverables...")
print("=" * 70)

# Calculate gap_pct here before export
gap_summary['gap_pct'] = ((gap_summary['demand'] - gap_summary['supply']) / (gap_summary['supply'] + 1) * 100).round(2)

export_summary = gap_summary[['parent_category', 'category', 'supply', 'demand', 
                               'gap_score', 'gap_status', 'gap_pct']].copy()
export_summary.columns = ['parent_category', 'subcategory', 'supply', 'demand', 
                          'gap_score', 'gap_status', 'gap_pct']

print("\n1. Exporting data summaries...")
csv_path, json_path = export_dataframe_to_files(
    export_summary, 
    output_base / "data", 
    "subcategory_gap_summary"
)
print(f"   CSV: {csv_path}")
print(f"   JSON: {json_path}")

print("\n2. Exporting interactive charts (HTML)...")
charts_dir = output_base / "interactive"

export_top15 = gap_summary.head(15).copy()
export_top15['category_display'] = export_top15['category'] + ' (' + export_top15['parent_category'] + ')'
export_top15['category'] = export_top15['category_display']

export_plotly_chart(create_supply_demand_chart(export_top15), 
                   charts_dir / "01_supply_vs_demand_subcategory.html", format='html')
export_plotly_chart(chart_severity, charts_dir / "02_gap_severity_subcategory.html", format='html')
export_plotly_chart(chart_heatmap, charts_dir / "03_gap_heatmap_subcategory.html", format='html')
export_plotly_chart(chart_ranking, charts_dir / "04_subcategory_rankings.html", format='html')
export_plotly_chart(chart_distribution, charts_dir / "05_gap_distribution_subcategory.html", format='html')
print(f"   Interactive charts saved to: {charts_dir}")

print("\n3. Exporting static images (PNG)...")
images_dir = output_base / "images"
export_plotly_chart(create_supply_demand_chart(export_top15), 
                   images_dir / "01_supply_vs_demand_subcategory.png", format='png', width=1200, height=700)
export_plotly_chart(chart_severity, images_dir / "02_gap_severity_subcategory.png", 
                   format='png', width=1200, height=max(700, len(gap_summary) * 15))
export_plotly_chart(chart_heatmap, images_dir / "03_gap_heatmap_subcategory.png", format='png', width=1200, height=600)
export_plotly_chart(chart_ranking, images_dir / "04_subcategory_rankings.png", format='png', width=1200, height=700)
export_plotly_chart(chart_distribution, images_dir / "05_gap_distribution_subcategory.png", format='png', width=1080, height=500)
print(f"   Static images saved to: {images_dir}")

print("\n4. Exporting insights...")
all_insights = {
    'supply_demand_analysis': insight_supply_demand,
    'severity_analysis': insight_severity,
    'heatmap_analysis': insight_heatmap,
    'ranking_analysis': insight_ranking
}
insights_path = export_insights_to_text(all_insights, output_base / "insights_subcategory.txt")
print(f"   Insights saved to: {insights_path}")

print("\n" + "=" * 70)
print("All subcategory-level deliverables exported successfully")

Exporting subcategory-level deliverables...

1. Exporting data summaries...
   CSV: ../results/visualizations_subcategory/data/subcategory_gap_summary.csv
   JSON: ../results/visualizations_subcategory/data/subcategory_gap_summary.json

2. Exporting interactive charts (HTML)...
   Interactive charts saved to: ../results/visualizations_subcategory/interactive

3. Exporting static images (PNG)...
   Static images saved to: ../results/visualizations_subcategory/images

4. Exporting insights...
   Insights saved to: ../results/visualizations_subcategory/insights_subcategory.txt

All subcategory-level deliverables exported successfully


---

##  Executive Summary 

In [24]:
executive_summary = generate_executive_summary(gap_summary)

print("=" * 70)
print("EXECUTIVE SUMMARY (SUBCATEGORY LEVEL)")
print("=" * 70)

print(f"\nCRITICAL GAPS ({executive_summary['critical_gaps']['count']} subcategories):")
print(f"  Most Severe: {executive_summary['critical_gaps']['most_severe']}")
print(f"    Parent Category: {gap_summary[gap_summary['category'] == executive_summary['critical_gaps']['most_severe']].iloc[0]['parent_category']}")
print(f"  Gap Score: {executive_summary['critical_gaps']['gap_score']:.2f}")
print(f"  Supply: {executive_summary['critical_gaps']['supply']:,} products")
print(f"  Demand: {executive_summary['critical_gaps']['demand']:,} units")
print(f"  Gap Severity: {executive_summary['critical_gaps']['gap_pct']:.1f}%")

print(f"\nWELL-BALANCED SUBCATEGORIES:")
print(f"  Example: {executive_summary['balanced_categories']['example']}")
print(f"    Parent Category: {gap_summary[gap_summary['category'] == executive_summary['balanced_categories']['example']].iloc[0]['parent_category']}")
print(f"  Gap Score: {executive_summary['balanced_categories']['gap_score']:.2f}")
print(f"  Supply: {executive_summary['balanced_categories']['supply']:,} products")
print(f"  Demand: {executive_summary['balanced_categories']['demand']:,} units")

print(f"\nOVERALL METRICS:")
print(f"  Total Subcategories: {executive_summary['overall_metrics']['total_categories']}")
print(f"  Parent Categories: {gap_summary['parent_category'].nunique()}")
print(f"  Average Gap Score: {executive_summary['overall_metrics']['avg_gap_score']:.2f}")
print(f"  Critical/High Gap Subcategories: {executive_summary['overall_metrics']['critical_high_count']} ({executive_summary['overall_metrics']['critical_high_pct']:.1f}%)")
print(f"  Total Supply: {executive_summary['overall_metrics']['total_supply']:,} units")
print(f"  Total Demand: {executive_summary['overall_metrics']['total_demand']:,} units")

print("\nTOP 5 PARENT CATEGORIES WITH MOST GAPS:")
category_gap_summary = gap_summary[
    gap_summary['gap_status'].isin(['Critical Gap', 'High Gap'])
].groupby('parent_category').agg({
    'category': 'count',
    'gap_score': ['mean', 'max']
}).round(2)
category_gap_summary.columns = ['critical_subcat_count', 'avg_gap', 'max_gap']
category_gap_summary = category_gap_summary.sort_values('critical_subcat_count', ascending=False)
for idx, (cat, row) in enumerate(category_gap_summary.head().iterrows(), 1):
    print(f"  {idx}. {cat}: {int(row['critical_subcat_count'])} critical subcategories (avg gap: {row['avg_gap']:.2f})")

print("\nRECOMMENDATIONS:")
print("\n  Immediate Actions:")
for rec in executive_summary['recommendations']['immediate']:
    print(f"    - {rec}")

print("\n  Medium-Term Strategy:")
for rec in executive_summary['recommendations']['medium_term']:
    print(f"    - {rec}")
print(f"    - Focus on parent categories with multiple high-gap subcategories")

print("\n  Strategic Initiatives:")
for rec in executive_summary['recommendations']['strategic']:
    print(f"    - {rec}")
print(f"    - Develop subcategory-specific product expansion roadmap")

print("\n" + "=" * 70)
print("Subcategory-level analysis complete and ready for presentation")
print("=" * 70)

EXECUTIVE SUMMARY (SUBCATEGORY LEVEL)

CRITICAL GAPS (10 subcategories):
  Most Severe: Bottles and Cages
    Parent Category: Accessories
  Gap Score: 3776.75
  Supply: 3 products
  Demand: 15,106 units
  Gap Severity: 377575.0%

WELL-BALANCED SUBCATEGORIES:
  Example: Mountain Frames
    Parent Category: Components
  Gap Score: 0.03
  Supply: 28 products
  Demand: 0 units

OVERALL METRICS:
  Total Subcategories: 37
  Parent Categories: 4
  Average Gap Score: 402.16
  Critical/High Gap Subcategories: 16 (43.2%)
  Total Supply: 293 units
  Total Demand: 84,174 units

TOP 5 PARENT CATEGORIES WITH MOST GAPS:
  1. Accessories: 8 critical subcategories (avg gap: 1402.20)
  2. Clothing: 6 critical subcategories (avg gap: 543.55)
  3. Bikes: 2 critical subcategories (avg gap: 152.00)

RECOMMENDATIONS:

  Immediate Actions:
    - expand supply: Bottles and Cages
    - launch campaigns in high-gap categories
    - investigate supply chain bottlenecks

  Medium-Term Strategy:
    - demand manag

---
## Deliverables Summary 

All outputs for the subcategory-level analysis have been generated and stored in the
results/visualizations_subcategory directory.


**Data Exports :**

CSV files containing parent category and subcategory fields along with all gap metrics.

JSON exports suitable for web applications or API integration.

Complete dataset covering all 37 subcategories with no missing records.

**Interactive Visualizations :**

Fully interactive HTML files with zoom, pan, and hover tooltips.

Subcategory labels include parent-category context for better clarity.

Ready for stakeholder presentations or embedding on websites.

**Static Images :**

High-resolution PNG images.

Figure sizes automatically adjust based on the number of subcategories.

Suitable for reports, documentation, and formal presentations.

**Insights :**

Clear textual analysis for each subcategory.

Executive summary highlighting issues across parent categories.

Practical, action-oriented recommendations based on supply-demand gaps.

**Key Benefits of Subcategory Analysis :**

More Granular Insights: Identify specific product lines that require attention.

Targeted Action: Pinpoint exactly which subcategories need supply expansion.

Better Resource Allocation: Focus investments on high-impact areas.

Parent-Category Awareness: Understand which main categories face the largest gaps.

Comprehensive View: 37 subcategories vs. only 4 main categories → about 9× more detail.

This subcategory-level visualization package delivers a detailed, actionable foundation for strategic decision-making in supply–demand gap management.


---